In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from bagging_classifier import CustomBaggingClassifier
from datasets import load_dataset

In [ ]:
X, y, numerical_columns, categorical_columns, label_encoder = load_dataset(name="adult")
#X, y, numerical_columns, categorical_columns, label_encoder = load_dataset(name="wine")

### Custom

In [ ]:
model = CustomBaggingClassifier(estimator=LinearSVC, sample_size=0.8, sample_size_features=0.8, n_estimators=5, categorical_features=categorical_columns)
#model = CustomBaggingClassifier(estimator=DecisionTreeClassifier, sample_size=0.8, sample_size_features=0.8, n_estimators=5)
#model = CustomBaggingClassifier(estimator=LogisticRegression, sample_size=0.8, sample_size_features=0.8, n_estimators=5)

In [ ]:
model.fit(X, y)

In [ ]:
cross_validate(model, X, y, cv=5, scoring=("accuracy"))

### scikit-learn

In [ ]:
def preprocess(X):
    numerical_features = list(set(X.columns) - set(categorical_columns))
    transformer = ColumnTransformer(
        [('ct', StandardScaler(), numerical_features),
         ("ohe", OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns)],
        remainder='passthrough')
    return transformer.fit_transform(X)

X_transformed = preprocess(X)

In [ ]:
model = RandomForestClassifier()
#model = BaggingClassifier(estimator=LinearSVC())
model.fit(X_transformed, y)
cross_validate(model, X_transformed, y, cv=5, scoring=("accuracy"))